In [ ]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy import linalg
import matplotlib.pyplot as plt
import random
import math
from IPython.display import display, clear_output
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from jupyterthemes import jtplot
import time
import scipy as scipy
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import csv

jtplot.style(theme = 'gruvboxd')

In [ ]:
# scenarios = 10
# total_time = 50
# sample_no = 30
# SLHL_C14 = 14.1
# SLHL_Be10 = 4.1
# SLHL_He3 = 120.5
# scaling_factor = 1.1
# stoch_base_ER = 0.1

In [ ]:
# max_depth = [350,1500]
# # dat_lin = np.arange(0,max_depth,1)
# rock_rho = 2.7
# CN_lambda = 160
# mu = rock_rho/ CN_lambda
# dt = 1
# MAT = 0
# T_AMP = 5

In [ ]:
# rock_rho = 2.7
# CN_lambda = 160
# mu = rock_rho / CN_lambda
# dt = 1

# # He diffusion inputs
# rho = 2.65 # quartz density
# nx = 100
# r = 0.1
# dx = (r / nx)
# x = np.arange(dx/2, r, dx)
# x_up = x + (dx/2)
# x_low = x - (dx/2)
# x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))]
# shell_mass = [(x_vol[i] * rho) for i in range(len(x_vol))]

# # Bedrock thermal properties
# alpha = 1.5e-6

In [ ]:
def ER_import(stoch_base_ER):
    read_in = open('D:/STEIN_paper/model_outputs/stochastic_erosion/erosion_sets/import_dates.csv')
    myreader = csv.reader(read_in)
    
    for row in myreader:
        mat_gen_dates = row[:]
    
    ERs = np.genfromtxt('D:/STEIN_paper/model_outputs/stochastic_erosion/erosion_sets/ER_list_' + str(stoch_base_ER) + '_' + str(mat_gen_dates[-1]) + '.csv',
                                  delimiter = ',')
        
    return ERs

In [ ]:
def CN_stoch_in_fxn(ERs_all):
    CN_import = np.empty((total_time, 4, scenarios))
    for i in range(scenarios):
        CN_import[:,:3,i] = np.genfromtxt('D:/STEIN_paper/model_outputs/stochastic_erosion/STO_C14_Be10_ER' + str(ERs_all[i]) + '_expmat.csv', 
                                          delimiter = ',')

        CN_import[:,3,i] = np.genfromtxt('D:/STEIN_paper/model_outputs/stochastic_erosion/STO_He3_ER' + str(ERs_all[i]) + '_surfmat.csv',
                              delimiter = ',')
        
    return CN_import

In [ ]:
def sampling_fxn(CN_import_matrix, sample_no, scenarios):
    P0_Be10 = SLHL_Be10 * scaling_factor
    P0_He3 = SLHL_Be10 * scaling_factor
    sampling_years = np.sort(np.random.choice(np.arange(int(total_time * 0.1),total_time),sample_no))
    sampling_mat = np.empty((sample_no, 5, scenarios))
    measured = np.empty((5,scenarios))
    
    for j in range(scenarios):
        
        sampling_14C = []
        sampling_Be = []
        sampling_rat = []
        sampling_3He = []
    
        for i in range(sample_no):           

            sampling_14C.append(CN_import_matrix[sampling_years[i],0,j])
            sampling_Be.append(CN_import_matrix[sampling_years[i],1,j])
            sampling_rat.append(CN_import_matrix[sampling_years[i],2,j])
            sampling_3He.append(CN_import_matrix[sampling_years[i],3,j])
        
        sampling_mat[:,0,j] = sampling_14C
        sampling_mat[:,1,j] = sampling_Be
        sampling_mat[:,2,j] = sampling_rat
        sampling_mat[:,3,j] = sampling_3He
        sampling_mat[:,4,j] = sampling_years 
        
        measured[0,j] = np.mean(sampling_mat[:,0,j])
        measured[1,j] = np.mean(sampling_mat[:,1,j])
        measured[2,j] = measured[0,j] / measured[1,j]
        measured[3,j] = np.mean(sampling_mat[:,4,j])
        measured[4,j] = (np.mean(sampling_mat[:,4,j]) / np.mean(sampling_mat[:,1])) * (P0_Be10 / P0_He3)
        
    measured_ER = [(160 / 2.62 * P0_Be10 / measured[1,j]) for j in range(scenarios)]
    
    return sampling_mat, measured, measured_ER  

In [ ]:
def generate_constant_comps(measured_ER):
    %run D:\STEIN_paper\model_outputs\scripts\C14_Be10_constant_erosion_for_stoch_compare.ipynb
    %run D:\STEIN_paper\model_outputs\scripts\He3_constant_erosion_for_stoch_compare.ipynb
    
    comp_mat = np.empty((total_time, 5, scenarios))
    
    shift_ER = False
    save_only_surf = True
    save_output = True    
    
    for i in range(scenarios):
        
        initial_ER = measured_ER[i]
        comp_mat[:,2,i], comp_mat[:,0,i], comp_mat[:,1,i] = CRN_comp_loop_fxn(total_time, initial_ER)[2:5] 
        comp_mat[:,3,i] = He3_comp_loop_fxn(total_time, initial_ER)
        comp_mat[:,4,i] = np.divide(comp_mat[:,3,i], comp_mat[:,1,i]) * P0_Be10 / P0_He3
        
    return comp_mat

In [ ]:
def compare_fxn():
    
    ERs_all = ER_import(stoch_base_ER)
    CN_import = CN_stoch_in_fxn(ERs_all)
    samples, measured, measured_ER = sampling_fxn(CN_import, sample_no, scenarios)
    const_comp_mat = generate_constant_comps(measured_ER)
    
    return const_comp_mat